In [1]:
import pandas as pd 
import numpy as np 
import os
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import re
import time

In [2]:
data_src = "./arxiv-metadata-oai-snapshot.json"

def get_meta_data():
    with open(data_src, 'r') as f:
        for line in f:
            yield line

In [3]:
metadata = get_meta_data()
for paper in metadata:
    for key, val in json.loads(paper).items():
        # print(f'{key}: {val}')
        print(key)
    break

id
submitter
authors
title
comments
journal-ref
doi
report-no
categories
license
abstract
versions
update_date
authors_parsed


create DF

In [4]:
NUMBER_FILTER = 2390080

In [5]:
def get_dataset_generator(path):
    with open(path, "r") as fp:
        for line in fp:
            row = json.loads(line)
            yield row

            
dataset_generator = get_dataset_generator(data_src)
print(type(dataset_generator))

<class 'generator'>


In [6]:
def create_dataframe(generator):
    ids = []
    submitters = []
    authors = []
    titles = []
    categories = []
    abstracts = []
    categories = []
    dates = []
    
    # dois = []
    # comments = []
    # journal_refs = []
    # report_nos = []
    # versions = []

    for row in generator:
        # if len(id) == NUMBER_FILTER:
        #     break
        
        ids.append(row["id"])
        submitters.append(row["submitter"])
        titles.append(row["title"])
        categories.append(row["categories"])
        abstracts.append(row["abstract"])
        authors.append(row["authors"])
        dates.append(row["update_date"])

        # comments.append(row["comments"])
        # dois.append(row["doi"])
        # journal_refs.append(row["journal-ref"])
        # report_nos.append(row["report-no"])
        # versions.append(row["versions"])
        
        
        
        
    return pd.DataFrame.from_dict({
        "id": ids,
        "submitter": submitters,
        "title": titles,
        "categories": categories,
        "abstract": abstracts, 
        "authors": authors,
        "date": dates,     
    })

t1 = time.time()
df_arxiv = create_dataframe(dataset_generator)
df_arxiv["date"] = pd.to_datetime(df_arxiv["date"])
t2 = time.time()

In [7]:
print(f"Elapsed time to create DF: {t2-t1}")
print(f"Len DF: {len(df_arxiv)}")

Elapsed time to create DF: 54.81331467628479
Len DF: 2390081


In [8]:
df_arxiv.head()

,id,submitter,title,categories,abstract,authors,date
0,0704.0001,Pavel Nadolsky,Calculation of prompt diphoton production cros...,hep-ph,A fully differential calculation in perturba...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",2008-11-26
1,0704.0002,Louis Theran,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"We describe a new algorithm, the $(k,\ell)$-...",Ileana Streinu and Louis Theran,2008-12-13
2,0704.0003,Hongjun Pan,The evolution of the Earth-Moon system based o...,physics.gen-ph,The evolution of Earth-Moon system is descri...,Hongjun Pan,2008-01-13
3,0704.0004,David Callan,A determinant of Stirling cycle numbers counts...,math.CO,We show that a determinant of Stirling cycle...,David Callan,2007-05-23
4,0704.0005,Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,math.CA math.FA,In this paper we show how to compute the $\L...,Wael Abu-Shammala and Alberto Torchinsky,2013-10-15


In [9]:
df_arxiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390081 entries, 0 to 2390080
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   submitter   object        
 2   title       object        
 3   categories  object        
 4   abstract    object        
 5   authors     object        
 6   date        datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 127.6+ MB


In [10]:
df_arxiv.isnull().sum()

id                0
submitter     15189
title             0
categories        0
abstract          0
authors           0
date              0
dtype: int64

In [11]:
print(len(df_arxiv.categories.unique()))
print(df_arxiv.categories.unique())

80625
['hep-ph' 'math.CO cs.CG' 'physics.gen-ph' ...
 'supr-con cond-mat.mtrl-sci cond-mat.supr-con'
 'supr-con cond-mat.mtrl-sci cond-mat.supr-con nlin.PS patt-sol'
 'supr-con cond-mat.supr-con quant-ph']


In [12]:
df_arxiv_v2 = df_arxiv.drop(['id', 'submitter','authors','date'], axis=1)
df_arxiv_v2.head()

,title,categories,abstract
0,Calculation of prompt diphoton production cros...,hep-ph,A fully differential calculation in perturba...
1,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"We describe a new algorithm, the $(k,\ell)$-..."
2,The evolution of the Earth-Moon system based o...,physics.gen-ph,The evolution of Earth-Moon system is descri...
3,A determinant of Stirling cycle numbers counts...,math.CO,We show that a determinant of Stirling cycle...
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,math.CA math.FA,In this paper we show how to compute the $\L...


In [13]:
print(f"Len DF: {len(df_arxiv)}")

Len DF: 2390081


In [14]:
df_arxiv_v2["abstract"][0]

'  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n'

In [15]:
# remove new line characters
df_arxiv_v2['abstract'] = df_arxiv_v2['abstract'].str.replace('\n', ' ')
df_arxiv_v2['title'] = df_arxiv_v2['title'].str.replace('\n', ' ')

In [16]:
df_arxiv_v2["abstract"][0]

'  A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events. '

In [17]:
df_arxiv_v2.head()

,title,categories,abstract
0,Calculation of prompt diphoton production cros...,hep-ph,A fully differential calculation in perturba...
1,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"We describe a new algorithm, the $(k,\ell)$-..."
2,The evolution of the Earth-Moon system based o...,physics.gen-ph,The evolution of Earth-Moon system is descri...
3,A determinant of Stirling cycle numbers counts...,math.CO,We show that a determinant of Stirling cycle...
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,math.CA math.FA,In this paper we show how to compute the $\L...


In [18]:
# # df_arxiv_v2['abstract'] = df_arxiv_v2['abstract'].str.replace("\$\([^)]*\)\$", ' ', regex=True)
# # df_arxiv_v2.head()

In [35]:
df_arxiv_v2.to_csv('arxiv_title_abs_v1.csv')

## 3) Filter for a category

In [19]:
print(len(df_arxiv_v2.categories.unique()))
print(df_arxiv_v2.categories.unique())

80625
['hep-ph' 'math.CO cs.CG' 'physics.gen-ph' ...
 'supr-con cond-mat.mtrl-sci cond-mat.supr-con'
 'supr-con cond-mat.mtrl-sci cond-mat.supr-con nlin.PS patt-sol'
 'supr-con cond-mat.supr-con quant-ph']


In [20]:
df_arxiv_CS = df_arxiv_v2[df_arxiv_v2["categories"].str.contains(" cs\.[A-Za-z0-9]+",regex=True, case=True)]

In [21]:
print(f"Len DF: {len(df_arxiv_CS)}")

Len DF: 299131


In [22]:
df_arxiv_CS.head()

,title,categories,abstract
1,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"We describe a new algorithm, the $(k,\ell)$-..."
45,A limit relation for entropy and channel capac...,quant-ph cs.IT math.IT,"In a quantum mechanical model, Diosi, Feldma..."
46,Intelligent location of simultaneously active ...,cs.NE cs.AI,The intelligent acoustic emission locator is...
49,Intelligent location of simultaneously active ...,cs.NE cs.AI,Part I describes an intelligent acoustic emi...
89,Real Options for Project Schedules (ROPS),cs.CE cond-mat.stat-mech cs.MS cs.NA physics.d...,Real Options for Project Schedules (ROPS) ha...


In [23]:
print(f"Len DF: {len(df_arxiv_CS.categories.unique())}")
df_arxiv_CS.categories

Len DF: 32930


1                                              math.CO cs.CG
45                                    quant-ph cs.IT math.IT
46                                               cs.NE cs.AI
49                                               cs.NE cs.AI
89         cs.CE cond-mat.stat-mech cs.MS cs.NA physics.d...
                                 ...                        
2388795                                       quant-ph cs.CC
2388828                                       quant-ph cs.CC
2388882                                       quant-ph cs.CR
2388963                                       quant-ph cs.CR
2389144    quant-ph chao-dyn cond-mat.mes-hall cs.CC nlin...
Name: categories, Length: 299131, dtype: object

In [24]:
df_arxiv_CS.to_csv('arxiv_title_abs_CS_v2.csv')

In [65]:
df_arxiv_CS_2000 = df_arxiv_CS.sample(2000)
print(f"Len DF: {len(df_arxiv_CS_2000)}")
df_arxiv_CS_2000.to_csv('arxiv_title_abs_CS_2000.csv')

Len DF: 2000
